In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6912,2024-10-20,Espanha Acb,07:30,Lleida,Manresa,2.12,1.73,164.5,1.83,1.91,1.5,1.95,1.79,4dDUotTq,0.471698,0.578035,0.546448,0.523560,0.049733,0.0,0.4,NaN,NaN,NaN,NaN,NaN,181.954,24.065273,0.132260,0.000,173.504,0.00,184.80,0.0,0.0,0.0,0.0,0.143258,0.030251,0.060501,0.00,0.000,inf,0.000000,0.0,0.000000,0.93,0.186,3.924731,0.411040,0.6,0.188960
6913,2024-10-20,Espanha Acb,07:30,Real Madrid,Basquet Girona,1.09,7.70,166.5,1.84,1.90,-15.5,1.97,1.78,Y9jc4lqB,0.917431,0.129870,0.543478,0.526316,0.047301,0.4,0.4,NaN,NaN,140.274,47.349454,0.337550,221.078,179.650865,0.812613,100.914,389.102,97.52,524.60,0.0,1.0,0.0,0.0,1.063476,0.022688,0.071653,0.51,0.102,0.882353,0.000000,0.0,0.000000,1.10,0.220,30.454545,0.000000,0.0,0.000000
6914,2024-10-20,Espanha Acb,12:00,Granada,Tenerife,2.68,1.48,159.5,1.90,1.90,3.5,2.04,1.72,0O6wpKcd,0.373134,0.675676,0.526316,0.526316,0.048810,0.6,0.6,NaN,NaN,260.888,145.642770,0.558258,169.682,35.598187,0.209794,169.992,154.394,140.60,158.40,0.0,0.0,0.0,0.0,0.407946,0.000000,0.120359,0.63,0.126,13.333333,0.000000,0.0,0.000000,1.91,0.382,1.256545,0.000000,0.0,0.000000
6915,2024-10-20,Espanha Acb,12:00,Valencia,MoraBanc Andorra,1.24,4.15,168.5,1.86,1.94,-10.5,1.99,1.76,b5ThDD2M,0.806452,0.240964,0.537634,0.515464,0.047415,0.8,0.8,NaN,NaN,177.290,40.077791,0.226058,154.624,43.069812,0.278545,149.470,323.726,235.62,227.24,0.0,0.0,0.0,0.0,0.763518,0.029773,0.086738,1.62,0.324,0.740741,0.708945,0.6,-0.108945,-5.00,-1.000,-3.150000,0.318014,0.1,-0.218014
6916,2024-10-20,Espanha Acb,13:30,Barcelona,Baskonia,1.27,3.83,169.5,1.81,1.93,-10.5,2.03,1.66,IkAP7iQq,0.787402,0.261097,0.552486,0.518135,0.048498,0.4,0.8,NaN,NaN,140.180,23.525238,0.167822,176.582,51.455268,0.291396,130.414,176.192,99.19,161.60,0.0,0.0,0.0,0.0,0.709880,0.045376,0.141805,-2.18,-0.436,-0.619266,0.794179,0.6,-0.194179,-3.29,-0.658,-4.300912,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,2024-10-20,México Lnbp,21:00,Soles,Halcones Rojos,1.30,3.31,168.5,1.81,1.85,-8.5,1.95,1.73,Y5M629vm,0.769231,0.302115,0.552486,0.540541,0.071346,0.4,0.0,NaN,NaN,139.220,33.300510,0.239193,NaN,NaN,NaN,139.220,0.000,152.88,183.16,0.0,0.0,0.0,0.0,0.616609,0.015456,0.084545,2.38,0.476,0.630252,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7072,2024-10-20,México Lnbp,22:30,Panteras,Freseros,1.24,3.77,175.5,1.83,1.83,-11.5,2.01,1.68,0zLeBq6K,0.806452,0.265252,0.546448,0.546448,0.071704,0.4,0.0,NaN,NaN,158.112,28.111260,0.177793,NaN,NaN,NaN,144.954,0.000,130.00,236.39,0.0,0.0,0.0,0.0,0.714164,0.000000,0.126474,0.04,0.008,30.000000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7073,2024-10-20,República Tcheca 1. Liga,11:30,Vysoka,Brno U23,5.72,1.12,158.5,1.83,1.83,11.5,2.00,1.72,M9ggGgRO,0.174825,0.892857,0.546448,0.546448,0.067682,0.0,0.0,NaN,NaN,NaN,NaN,NaN,112.812,16.792341,0.148852,0.000,0.000,0.00,121.04,0.0,0.0,0.0,0.0,0.951079,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7074,2024-10-20,Romênia Divisão A,13:00,Dinamo Bucharest,Valcea,1.80,1.92,158.5,1.86,1.80,-1.5,1.93,1.78,pxltm2Q0,0.555556,0.520833,0.537634,0.555556,0.076389,0.6,1.0,NaN,NaN,151.348,18.560779,0.122636,231.694,182.531517,0.787813,147.832,288.782,125.06,180.48,0.0,0.0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
